In [1]:
import numpy as np
import h5py
import pdb
import re

/home/cody/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
line_number = None
data = []
with open('../data/combined_data_1.txt') as openfileobject:
    for line in openfileobject:
        if ":" in line:
            line_number = line.split(":")
            line_number = "".join(line_number)
        else:
            rating = re.split('\W+',line) 
            date = "".join(rating[2:-1])
            rating.pop()
            rating.pop()
            rating.pop()
            rating.pop()
            rating.append(date)
            rating.insert(0, int(line_number))
            rating = list(map(int, rating))
            data.append(np.array(rating))

data = np.array(data)

line_number = None
data2 = []
with open('../data/combined_data_2.txt') as openfileobject:
    for line in openfileobject:
        if ":" in line:
            line_number = line.split(":")
            line_number = "".join(line_number)
        else:
            rating = re.split('\W+',line) 
            date = "".join(rating[2:-1])
            rating.pop()
            rating.pop()
            rating.pop()
            rating.pop()
            rating.append(date)
            rating.insert(0, int(line_number))
            rating = list(map(int, rating))
            data2.append(np.array(rating))

data2 = np.array(data2)

line_number = None
data3 = []
with open('../data/combined_data_3.txt') as openfileobject:
    for line in openfileobject:
        if ":" in line:
            line_number = line.split(":")
            line_number = "".join(line_number)
        else:
            rating = re.split('\W+',line) 
            date = "".join(rating[2:-1])
            rating.pop()
            rating.pop()
            rating.pop()
            rating.pop()
            rating.append(date)
            rating.insert(0, int(line_number))
            rating = list(map(int, rating))
            data3.append(np.array(rating))

data3 = np.array(data3)

line_number = None
data4 = []
with open('../data/combined_data_4.txt') as openfileobject:
    for line in openfileobject:
        if ":" in line:
            line_number = line.split(":")
            line_number = "".join(line_number)
        else:
            rating = re.split('\W+',line) 
            date = "".join(rating[2:-1])
            rating.pop()
            rating.pop()
            rating.pop()
            rating.pop()
            rating.append(date)
            rating.insert(0, int(line_number))
            rating = list(map(int, rating))
            data4.append(np.array(rating))

data4 = np.array(data4)

combined_data = np.concatenate([data, data2, data3, data4], axis=0)
del data
del data2
del data3
del data4

In [3]:
hf = h5py.File('../data/Full_NF.h5', 'w')

In [4]:
hf.create_dataset('dataset_1', data=combined_data)

<HDF5 dataset "dataset_1": shape (100480507, 4), type "<i8">

In [5]:
del combined_data
hf.close()

In [20]:
dset = h5py.File('../data/Full_NF.h5', 'r')

In [21]:
dset = dset["dataset_1"]

In [22]:
dset = dset[:]

In [10]:
user_list = list(set(dset[:,1]))

In [11]:
del dset

In [12]:
user_list[0]

1048577

In [13]:
len(user_list)

480189

In [14]:
user_list.sort()

In [15]:
user_dictionary = {i: user for user, i in enumerate(user_list)}

In [16]:
import pickle
with open('../data/name_one_hot_dict.pickle', 'wb') as handle:
    pickle.dump(user_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
del user_dictionary

In [19]:
import statistics

In [23]:
dset[0]

array([       1,  1488844,        3, 20050906])

In [31]:
movie = 1
rating_list = []
mean_movie = {}
for rating in dset:
    if rating[0] == movie:
        rating_list.append(rating[2])
    else:
        mean_movie[movie] = statistics.mean(rating_list)
        rating_list = [rating[2]]
        movie += 1

mean_movie[movie] = statistics.mean(rating_list)
    

In [32]:
import pickle
with open('../data/mean_rating.pickle', 'wb') as handle:
    pickle.dump(mean_movie, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
movie = 1
rating_list = []
median_movie = {}
for rating in dset:
    if rating[0] == movie:
        rating_list.append(rating[2])
    else:
        median_movie[movie] = statistics.median(rating_list)
        rating_list = [rating[2]]
        movie += 1

median_movie[movie] = statistics.median(rating_list)

In [38]:
with open('../data/median_rating.pickle', 'wb') as handle:
    pickle.dump(median_movie, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
movie = 1
rating_list = []
var_movie = {}
for rating in dset:
    if rating[0] == movie:
        rating_list.append(rating[2])
    else:
        var_movie[movie] = statistics.variance(rating_list)
        rating_list = [rating[2]]
        movie += 1

var_movie[movie] = statistics.variance(rating_list)

In [40]:
with open('../data/movie_var.pickle', 'wb') as handle:
    pickle.dump(var_movie, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
mean_movie[2]

3

In [1]:
import h5py
import random

hf = h5py.File('../data/Full_NF.h5', 'r')
dset = hf["dataset_1"]
dset = dset[:]
movie_dict = {}
user_dict = {}

print("Data loaded from file")

for record in dset:
    if movie_dict.get(record[0]) != None:
        movie_dict[record[0]].append((record[1], record[2]))
    else:
        movie_dict[record[0]] = [(record[1], record[2])]
        
print("Movie dict created")
x_train = []
y_train = []
window_max = 5

for movie in movie_dict:
    users = [u[0] for u in movie_dict[movie] if u[1] > 2]
    if len(users) > 1:
        for user in users:
            window_size = min(window_max, len(users)  - 1)
            window = []
            while len(window) < window_size:
                x = random.choice(users)
                if x != user and x not in window:
                    window.append(x)
            
            y_train.append(user)
            x_train.append(window)
    else:
        y_train.append(users[0])
        x_train.append(users[0])
        
print("writing to tsv")

with open('../data/user_embedding_5_thresh_train.tsv', 'w') as oFile:
    for i in range(len(y_train)):
        oFile.write(str(y_train[i]) + '\t' + str(x_train[i]) + '\n')
        
hf.close()
print("finished")

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data loaded from file
Movie dict created
writing to tsv
finished


In [1]:
import pickle
with open('../data/name_one_hot_dict.pickle', 'rb') as handle:
    name_one_hot = pickle.load(handle)

In [2]:
name_one_hot

{1048577: 190479,
 1048579: 190480,
 6: 0,
 7: 1,
 8: 2,
 10: 3,
 2097163: 380134,
 1048592: 190482,
 2446680: 443466,
 1048594: 190483,
 2621443: 475114,
 1048600: 190484,
 25: 4,
 2097181: 380135,
 1048606: 190485,
 2053057: 372172,
 33: 5,
 2097186: 380137,
 2446683: 443468,
 1048612: 190486,
 42: 6,
 1048619: 190487,
 349533: 63440,
 1048624: 190488,
 1048629: 190489,
 1048633: 190490,
 2097210: 380139,
 59: 7,
 1048586: 190481,
 2097217: 380142,
 2621451: 475115,
 1048647: 190492,
 79: 8,
 2097232: 380143,
 83: 9,
 1048660: 190493,
 87: 10,
 2097242: 380145,
 1048668: 190494,
 94: 11,
 97: 12,
 2097252: 380148,
 2097253: 380149,
 2097254: 380150,
 2097256: 380151,
 2097258: 380152,
 2097260: 380153,
 2479051: 449330,
 2097264: 380154,
 2097267: 380155,
 116: 13,
 2097269: 380156,
 2367301: 428885,
 2446697: 443473,
 2409219: 436649,
 1048699: 190495,
 1048700: 190496,
 2097277: 380159,
 126: 14,
 1747648: 317083,
 130: 15,
 131: 16,
 1048708: 190497,
 133: 17,
 134: 18,
 1048711: 

In [4]:
inverse_name_one_hot = {}
for name in name_one_hot:
    inverse_name_one_hot[name_one_hot[name]] = name

In [5]:
inverse_name_one_hot

{0: 6,
 1: 7,
 2: 8,
 3: 10,
 4: 25,
 5: 33,
 6: 42,
 7: 59,
 8: 79,
 9: 83,
 10: 87,
 11: 94,
 12: 97,
 13: 116,
 14: 126,
 15: 130,
 16: 131,
 17: 133,
 18: 134,
 19: 142,
 20: 149,
 21: 158,
 22: 164,
 23: 168,
 24: 169,
 25: 177,
 26: 178,
 27: 183,
 28: 188,
 29: 189,
 30: 192,
 31: 195,
 32: 199,
 33: 201,
 34: 215,
 35: 231,
 36: 242,
 37: 247,
 38: 248,
 39: 250,
 40: 261,
 41: 265,
 42: 266,
 43: 267,
 44: 268,
 45: 283,
 46: 291,
 47: 296,
 48: 298,
 49: 299,
 50: 301,
 51: 302,
 52: 304,
 53: 305,
 54: 307,
 55: 308,
 56: 310,
 57: 312,
 58: 314,
 59: 330,
 60: 331,
 61: 333,
 62: 352,
 63: 358,
 64: 363,
 65: 368,
 66: 369,
 67: 379,
 68: 383,
 69: 384,
 70: 385,
 71: 392,
 72: 413,
 73: 416,
 74: 424,
 75: 437,
 76: 439,
 77: 440,
 78: 442,
 79: 453,
 80: 462,
 81: 466,
 82: 470,
 83: 471,
 84: 477,
 85: 478,
 86: 479,
 87: 481,
 88: 485,
 89: 490,
 90: 491,
 91: 492,
 92: 495,
 93: 508,
 94: 515,
 95: 517,
 96: 527,
 97: 529,
 98: 536,
 99: 540,
 100: 544,
 101: 546,
 102

In [6]:
with open('../data/inverse_name_one_hot_dict.pickle', 'wb') as oFile:
    pickle.dump(inverse_name_one_hot, oFile, protocol=pickle.HIGHEST_PROTOCOL)